<a href="https://colab.research.google.com/github/xxotto/covid19-peru/blob/main/Notebooks/3_DataFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obteniendo más información de cada caso positivo por COVID-19 en Perú

En este notebook se busca obtener un dataset de todos los **casos positivos** reportados por COVID-19, y agregar información sobre fallecimiento y vacunación recibida (mediante **"UUID o id_persona"**). Es decir se busca unir las 3 bases de datos, de casos positivos, fallecimientos y vacunación recibida en una sola.

Se considera como **vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## 1. Preparación

### 1.1 Descargar fuentes de datos

Las fuentes de datos pueden ser descargadas a través de la [Plataforma Nacional de Datos Abiertos](https://www.datosabiertos.gob.pe/) del Gobierno del Perú.

1. [Fallecidos por COVID-19](https://www.datosabiertos.gob.pe/dataset/fallecidos-por-covid-19-ministerio-de-salud-minsa)
2. [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion)
3. [Casos positivos por COVID-19](https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa)
4. [Centros de vacunación](https://www.datosabiertos.gob.pe/dataset/centros-de-vacunacion)
5. [Códigos equivalentes de UBIGEO del Perú](https://www.datosabiertos.gob.pe/dataset/codigos-equivalentes-de-ubigeo-del-peru)

**Nota 1:** En este caso todos los csv se encuentran en google drive, usted puede descargarlos desde los enlaces correspondientes y colocarlos en la dirección de RawData. Una vez realizado esto saltar al paso 2.

**Nota 2:** Puede que existan cambios en las bases de datos usadas ya que son actualizadas a diario (actualmente), por tanto cambios en el código pueden ser necesarios más adelante.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# Cambia nuestro directorio actual al directorio de Notebooks (donde se ejecutaría 3_DataFiltering.ipynb)
%cd /content/drive/MyDrive/GitHub/covid19-peru/Notebooks/
%ls

Mounted at /content/drive/
/content/drive/MyDrive/GitHub/covid19-peru/Notebooks
0_ExploratoryDataAnalysis.ipynb  3_DataFiltering.ipynb  6_DataFiltering.ipynb
1_DataFiltering.ipynb            4_DataFiltering.ipynb  DataProblems.ipynb
2_DataFiltering.ipynb            5_DataFiltering.ipynb  functions.py


### 1.2 Instalar librerías necesarias

In [ ]:
!pip install epiweeks

## 2. Lectura y Procesamiento

### 2.2 Cargar Librerías

In [ ]:
import pandas as pd
import numpy as np
from epiweeks import Week
from datetime import datetime

### 2.3 Diccionarios para codificar variables

In [ ]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS":1,
    "ANCASH":2,
    "APURIMAC":3,   
    "AREQUIPA":4,
    "AYACUCHO":5,   
    "CAJAMARCA":6,
    "CALLAO":7,
    "CUSCO":8,
    "HUANCAVELICA":9,	
    "HUANUCO":10,	
    "ICA":11,	
    "JUNIN":12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA":15,
    "LORETO":16,
    "MADRE DE DIOS":17,
    "MOQUEGUA":18,	
    "PASCO":19,
    "PIURA":20,
    "PUNO":21,
    "SAN MARTIN":22,
    "TACNA":23,
    "TUMBES":24,
    "UCAYALI":25
}

### 2.4 Dataset de casos positivos

#### 2.4.1 Leer dataset

In [ ]:
# Leer dataset
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'id_persona', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 'FECHA_RESULTADO': 'float64'})

In [ ]:
df_cas.head()

,DEPARTAMENTO,METODODX,EDAD,SEXO,FECHA_RESULTADO,id_persona
0,CALLAO,AG,27.0,MASCULINO,20220131.0,24662494.0
1,UCAYALI,AG,39.0,FEMENINO,20220127.0,24662689.0
2,LIMA,PR,25.0,MASCULINO,20201217.0,24662153.0
3,AREQUIPA,PCR,27.0,MASCULINO,20220120.0,24662155.0
4,ICA,PR,20.0,FEMENINO,20200822.0,24662175.0


In [ ]:
total_casos_reportados = len(df_cas)  # TOTAL de casos positivos
total_casos_reportados

3534686

#### 2.4.2 Tratamiento de datos

In [ ]:
df_cas.isnull().sum()  # Total de 'missing values' por columna

DEPARTAMENTO         321
METODODX               0
EDAD                 347
SEXO                   1
FECHA_RESULTADO     2023
id_persona         67222
dtype: int64

In [ ]:
df_cas = df_cas.dropna()  # Remover casos con información faltante
df_cas.isnull().sum()  # Total de 'missing values' por columna

DEPARTAMENTO       0
METODODX           0
EDAD               0
SEXO               0
FECHA_RESULTADO    0
id_persona         0
dtype: int64

In [ ]:
len(df_cas)  # TOTAL de casos positivos sin 'missing values'

3465000

In [ ]:
df_cas['DEPARTAMENTO'].value_counts().sort_index()  # Analizar columna DEPARTAMENTO

AMAZONAS           43249
ANCASH            122439
APURIMAC           38468
AREQUIPA          209268
ARICA                  1
AYACUCHO           47602
CAJAMARCA          94807
CALLAO            141864
CUSCO             114959
HUANCAVELICA       25410
HUANUCO            51374
ICA               101600
JUNIN             124181
LA LIBERTAD       143315
LAMBAYEQUE        104176
LIMA             1558569
LORETO             56750
MADRE DE DIOS      17374
MOQUEGUA           47630
PASCO              23673
PIURA             155048
PUNO               64770
SAN MARTIN         61131
TACNA              52855
TUMBES             26476
UCAYALI            38011
Name: DEPARTAMENTO, dtype: int64

In [ ]:
# Remover valores erróneos
rows_to_delete = df_cas[df_cas['DEPARTAMENTO'] == 'ARICA'].index
df_cas.drop(rows_to_delete, inplace = True)

In [ ]:
df_cas['METODODX'].value_counts().sort_index()  # Analizar columna METODODX

AG     1479496
PCR    1045382
PR      940121
Name: METODODX, dtype: int64

In [ ]:
# Analizar columna EDAD
edadCasos = df_cas['EDAD'].value_counts()

print('Edad (años): no_casos\n')
lst = []

for idx, edad in enumerate(edadCasos):
    index = edadCasos.index
    lst.append(f'{index[idx]:6}: {edad}   ')
    
lst = sorted(lst)
chunks = [lst[x:x+25] for x in range(0, len(lst), 25)]
res = pd.DataFrame.from_records(chunks).transpose()

print(res.to_string(index=False, header=False))

Edad (años): no_casos

    0.0: 5974      25.0: 79883      50.0: 54575      75.0: 14837       -79.0: 1   
    1.0: 8161      26.0: 82896      51.0: 52357      76.0: 13483     100.0: 157   
    2.0: 6608      27.0: 84990      52.0: 51051      77.0: 12538     101.0: 112   
    3.0: 6518      28.0: 87786      53.0: 50004      78.0: 11475      102.0: 76   
    4.0: 6735      29.0: 86784      54.0: 49424      79.0: 10277      103.0: 43   
    5.0: 7586      30.0: 83913      55.0: 47310       80.0: 9576      104.0: 20   
    6.0: 8642      31.0: 83924      56.0: 45040       81.0: 8955      105.0: 16   
    7.0: 9764      32.0: 86725      57.0: 43038       82.0: 8124      106.0: 11   
   8.0: 10877      33.0: 88587      58.0: 40962       83.0: 7474       107.0: 6   
   9.0: 11739      34.0: 82752      59.0: 39454       84.0: 6694       108.0: 8   
  10.0: 13308      35.0: 78892      60.0: 38264       85.0: 5986       109.0: 3   
  11.0: 15433      36.0: 76507      61.0: 36914       86.0: 5243

In [ ]:
# Remover valores erróneos
rows_to_delete = df_cas[(df_cas['EDAD'] < 0) | (df_cas['EDAD'] > 122)].index
df_cas.drop(rows_to_delete, inplace = True)

In [ ]:
df_cas['SEXO'].value_counts().sort_index()  # Analizar columna SEXO

FEMENINO     1750322
MASCULINO    1714672
Name: SEXO, dtype: int64

In [ ]:
df_cas['FECHA_RESULTADO'].nunique()  # Analizar columna FECHA_RESULTADO (Número de días con reportes de casos)

733

In [ ]:
personas_covid = df_cas['id_persona'].nunique()  # Analizar columna id_persona
personas_covid

3247356

In [ ]:
print(f'TOTAL de casos reportados: {total_casos_reportados}')
print(f'TOTAL de registros (id_persona) únicos: {personas_covid}')
print('Esta diferencia se debe a reinfecciones de COVID-19 en miles de personas')

TOTAL de casos reportados: 3534686
TOTAL de registros (id_persona) únicos: 3247356
Esta diferencia se debe a reinfecciones de COVID-19 en miles de personas


In [ ]:
# TOTAL de casos positivos sin valores erróneos
len(df_cas)

3464994

In [ ]:
# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)

df_cas = df_cas[['id_persona','fecha_resultado', 'metododx', 'sexo', 'edad', 'departamento']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

In [ ]:
# Codificar variables y cambiar 'data types'
df_cas['sexo'].replace(dic_sex, inplace=True)
df_cas['departamento'] = df_cas['departamento'].map(dic_dep)
df_cas['metododx'].replace(dic_met, inplace=True)

In [ ]:
# Cambiar 'data type' para reducir tamaño
df_cas = df_cas.astype({'id_persona': 'int32', 'fecha_resultado': 'int32', 'edad':'int8', 'departamento': 'int8',
                        'metododx': 'int8', 'sexo': 'int8'})

In [ ]:
df_cas.head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento
0,34978840,20200306,2,1,25,15
1,12149681,20200307,2,1,25,15
2,10629488,20200307,2,1,51,15
3,12211040,20200307,2,1,7,15
4,7800643,20200307,2,2,74,15


#### 2.4.3 Eliminar registros duplicados

Se decide eliminar casos de registros duplicados. Es decir eliminar del dataset todas las reinnfecciones de COVID-19 y conservar únicamente la última reinfección en caso de registros duplicados en id_persona.

In [ ]:
# Ordenar de acuerdo al id_de persona y fecha de resultado
df_cas = df_cas.sort_values(by = ['id_persona', 'fecha_resultado'])

In [ ]:
# Vemos algunos de los registros duplicados
df_cas[df_cas.id_persona.duplicated() == True].head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento
3214360,47,20220129,2,1,62,25
3214675,89,20220129,2,2,74,25
1726290,227,20210421,3,1,81,25
1923371,324,20210525,3,2,54,25
3164585,555,20220127,2,2,55,25


In [ ]:
# Comprobamos duplicados en el dataset principal de casos (df_cas)
df_cas[df_cas['id_persona'] == 47]

,id_persona,fecha_resultado,metododx,sexo,edad,departamento
330482,47,20200711,1,1,61,25
3214360,47,20220129,2,1,62,25


En este último caso el **id_persona** número **47** dió positivo a COVID-19 dos veces. La primera el 2020-07-11 y la segunda en 2022-01-29. 

Para el motivo del estudio se necesita información de UNA SOLA PERSONA, por lo que se decide eliminar estos casos de reinfecciones y conservar solamente la última vez que dió positivo. 

Estos duplicados representan varios contagios de la misma persona, sin embargo se toman como un nuevo contagio en la base de datos original.

In [ ]:
# Eliminamos los duplicados
df_cas = df_cas.drop_duplicates('id_persona', keep='last') \
               .sort_values(by = ['fecha_resultado']) \
               .reset_index(drop = True)

In [ ]:
df_cas.head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento
0,13708801,20200309,2,2,15,10
1,2020432,20200310,2,1,46,15
2,7979960,20200311,2,2,21,15
3,5493803,20200311,2,2,28,15
4,35668643,20200312,2,2,39,15


In [ ]:
# TOTAL de casos positivos con un id único para cada persona (sin registros duplicados)
len(df_cas)

3247356

### 2.5 Dataset de fallecidos

#### 2.5.1 Leer dataset

In [ ]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'UUID', 'SEXO', 'EDAD_DECLARADA', 'DEPARTAMENTO']
df_fal = pd.read_csv(url_fal, sep = ';', usecols= col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32'}, low_memory=False)

In [ ]:
df_fal.head()

,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,DEPARTAMENTO,UUID
0,20210611,21,MASCULINO,LAMBAYEQUE,24833991.0
1,20210317,45,MASCULINO,PIURA,24761117.0
2,20210602,62,FEMENINO,ICA,24767070.0
3,20210703,75,MASCULINO,AREQUIPA,24751741.0
4,20210506,66,FEMENINO,LIMA,24671820.0


In [ ]:
len(df_fal)  # TOTAL de fallecidos por COVID-19

211423

#### 2.5.2 Tratamiento de datos

In [ ]:
df_fal.isnull().sum()  # Total de 'missing values' por columna

FECHA_FALLECIMIENTO       0
EDAD_DECLARADA            0
SEXO                      0
DEPARTAMENTO              0
UUID                   3885
dtype: int64

In [ ]:
df_fal = df_fal.dropna()  # Remover casos con información faltante
df_fal.isnull().sum()  # Total de 'missing values' por columna

FECHA_FALLECIMIENTO    0
EDAD_DECLARADA         0
SEXO                   0
DEPARTAMENTO           0
UUID                   0
dtype: int64

In [ ]:
len(df_fal)  # TOTAL de casos positivos sin 'missing values'

207538

In [ ]:
# Renombrar columnas
df_fal.rename(columns = {
    'FECHA_FALLECIMIENTO':'fecha_fallecimiento',
    'UUID':'id_persona',
    'SEXO': 'sexo', 
    'EDAD_DECLARADA': 'edad',
    'DEPARTAMENTO': 'departamento'
    }, inplace=True)

In [ ]:
# Analizar columna EDAD
edadCasos = df_fal['edad'].value_counts()

print('Edad (años): no_casos\n')
lst = []

for idx, edad in enumerate(edadCasos):
    index = edadCasos.index
    lst.append(f'{index[idx]:6}: {edad}   ')
    
lst = sorted(lst)
chunks = [lst[x:x+25] for x in range(0, len(lst), 25)]
res = pd.DataFrame.from_records(chunks).transpose()

print(res.to_string(index=False, header=False))

Edad (años): no_casos

      0: 90         25: 241        50: 2564        75: 5061       100: 126   
     1: 121         26: 267        51: 2721        76: 4926        101: 83   
      2: 71         27: 331        52: 2825        77: 4717        102: 51   
      3: 50         28: 317        53: 3111        78: 4570        103: 29   
      4: 52         29: 337        54: 3460        79: 4205        104: 18   
      5: 42         30: 402        55: 3727        80: 4303         105: 6   
      6: 45         31: 410        56: 3751        81: 3986        106: 12   
      7: 49         32: 561        57: 4028        82: 4048         107: 2   
      8: 44         33: 611        58: 4071        83: 3750         108: 6   
      9: 51         34: 639        59: 4329        84: 3618         109: 2   
     10: 53         35: 651        60: 4634        85: 3413         110: 2   
     11: 52         36: 685        61: 5096        86: 3047         113: 2   
     12: 46         37: 836        62: 49

In [ ]:
df_fal['sexo'].value_counts().sort_index()  # Analizar columna SEXO

FEMENINO      75950
MASCULINO    131588
Name: sexo, dtype: int64

In [ ]:
# Reemplazar sexo con un code
df_fal['sexo'].replace(dic_sex, inplace=True)
df_fal['sexo'] = df_fal['sexo'].astype('int8')

In [ ]:
df_fal['departamento'].value_counts().sort_index()  # Analizar columna DEPARTAMENTO

AMAZONAS          1296
ANCASH            7014
APURIMAC          1611
AREQUIPA         10140
AYACUCHO          2285
CAJAMARCA         4387
CALLAO           10221
CUSCO             5074
HUANCAVELICA      1249
HUANUCO           2827
ICA               8830
JUNIN             7348
LA LIBERTAD      10701
LAMBAYEQUE        8724
LIMA             91290
LORETO            4021
MADRE DE DIOS      833
MOQUEGUA          1591
PASCO             1069
PIURA            12555
PUNO              4645
SAN MARTIN        3086
TACNA             2094
TUMBES            1664
UCAYALI           2983
Name: departamento, dtype: int64

In [ ]:
# Codificar variable departamento y cambiar 'data types'
df_fal['departamento'] = df_fal['departamento'].map(dic_dep)
df_fal['departamento'] = df_fal['departamento'].astype('int8')

# Nombrar a todos los casos de fallecimientos con un identificador
df_fal['fallecido'] = 1

# Cambiar de data type para reducir tamaño
df_fal = df_fal.astype({'id_persona': 'int32', 'fallecido': 'int8', 'sexo': 'int8', 'edad': 'int8',
                        'fecha_fallecimiento': 'int32'})

# Transformar a formato fecha para operar los días más adelante
df_fal['fecha_fal'] = pd.to_datetime(df_fal['fecha_fallecimiento'], format = '%Y%m%d')

# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha_fal'].apply(lambda date : Week.fromdate(date).weektuple()).tolist()

In [ ]:
df_fal.head()

,fecha_fallecimiento,edad,sexo,departamento,id_persona,fallecido,fecha_fal,year,epi_week
0,20210611,21,1,14,24833991,1,2021-06-11,2021,23
1,20210317,45,1,20,24761117,1,2021-03-17,2021,11
2,20210602,62,2,11,24767070,1,2021-06-02,2021,22
3,20210703,75,1,4,24751741,1,2021-07-03,2021,26
4,20210506,66,2,15,24671820,1,2021-05-06,2021,18


#### 2.4.3 Eliminar registros duplicados

Existen registros duplicados en el dataset de fallecidos, probablemente errores al momento de ingresar la información.

In [ ]:
# Ordenar de acuerdo al id_de persona y fecha de fallecimiento
df_fal = df_fal.sort_values(by = ['id_persona', 'fecha_fallecimiento'])

In [ ]:
# Vemos algunos de los registros duplicados
df_fal[df_fal.id_persona.duplicated() == True].head()

,fecha_fallecimiento,edad,sexo,departamento,id_persona,fallecido,fecha_fal,year,epi_week
141968,20210510,63,2,11,946577,1,2021-05-10,2021,19
147617,20200625,45,1,15,1378132,1,2020-06-25,2020,26
146507,20210407,60,1,15,1581994,1,2021-04-07,2021,14
149171,20210317,61,2,15,1676066,1,2021-03-17,2021,11
150571,20210323,64,1,15,1785991,1,2021-03-23,2021,12


In [ ]:
# Comprobamos duplicados en el dataset principal de fallecidos (df_fal)
df_fal[df_fal['id_persona'] == 946577]

,fecha_fallecimiento,edad,sexo,departamento,id_persona,fallecido,fecha_fal,year,epi_week
141966,20210510,73,2,11,946577,1,2021-05-10,2021,19
141968,20210510,63,2,11,946577,1,2021-05-10,2021,19


In [ ]:
# Eliminamos los duplicados
df_fal = df_fal.drop_duplicates('id_persona', keep='last') \
               .sort_values(by = ['fecha_fallecimiento']) \
               .reset_index(drop = True)

In [ ]:
df_fal.head()

,fecha_fallecimiento,edad,sexo,departamento,id_persona,fallecido,fecha_fal,year,epi_week
0,20200303,63,1,22,36527082,1,2020-03-03,2020,10
1,20200304,85,1,22,36401995,1,2020-03-04,2020,10
2,20200307,24,2,16,36853977,1,2020-03-07,2020,10
3,20200308,76,2,8,14179222,1,2020-03-08,2020,11
4,20200314,4,1,15,36614476,1,2020-03-14,2020,11


In [ ]:
# TOTAL de fallecidos con un id único para cada persona (sin registros duplicados)
len(df_fal)

207468

### 2.6 Buscar en el dataset de casos positivos quienes fallecieron y cuándo

In [ ]:
# Unir el dataframe de casos positivos con el dataset de fallecidos mediante 'id_persona'
df = df_cas.merge(df_fal, on = 'id_persona', how = 'outer')

df['fallecido'].fillna(0, inplace=True)  # Llenar con 0 las personas que no fallecieron en el dataset de casos positivos
df['fallecido'] = df['fallecido'].astype('int8')  # Cambiar 'data type' para reducir tamaño

In [ ]:
len(df)  # Total de casos de casos positivos + información de fallecimiento

3353127

In [ ]:
len(df[df.fallecido == 1])  # Número de fallecidos en el dataset de casos postivos

207468

#### 2.6.1 Unir información de casos positivos + información sobre fallecimiento

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo_x,edad_x,departamento_x,fecha_fallecimiento,edad_y,sexo_y,departamento_y,fallecido,fecha_fal,year,epi_week
0,13708801,20200309.0,2.0,2.0,15.0,10.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
1,2020432,20200310.0,2.0,1.0,46.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
2,7979960,20200311.0,2.0,2.0,21.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
3,5493803,20200311.0,2.0,2.0,28.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
4,35668643,20200312.0,2.0,2.0,39.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN


Tenemos algunas columnas duplicadas ya que son información del dataset de casos positivos (sexo_x, edad_x, departamento_x) y la otra información del dataset de fallecidos (sexo_y, edad_y, departamento_y). 

En este caso podemos ver que pertenecen a la misma persona ya que fueron unidos mediante el registro único de cada persona (id_persona). Procedemos a unir estas columnas duplicadas para completar información faltante.

In [ ]:
df[['id_persona', 'sexo_x', 'edad_x', 'departamento_x',
    'sexo_y', 'edad_y', 'departamento_y', 'fallecido']][df['fallecido'] == 1].head()

,id_persona,sexo_x,edad_x,departamento_x,sexo_y,edad_y,departamento_y,fallecido
5,36611013,1.0,78.0,15.0,1.0,78.0,15.0,1
9,36889651,2.0,75.0,15.0,2.0,75.0,15.0,1
51,36450848,1.0,96.0,15.0,1.0,96.0,15.0,1
62,36539210,1.0,71.0,7.0,1.0,71.0,15.0,1
106,36818328,2.0,63.0,15.0,2.0,63.0,15.0,1


In [ ]:
df.isnull().sum()  # Verificamos 'missing values'

id_persona                   0
fecha_resultado         105771
metododx                105771
sexo_x                  105771
edad_x                  105771
departamento_x          105771
fecha_fallecimiento    3145659
edad_y                 3145659
sexo_y                 3145659
departamento_y         3145659
fallecido                    0
fecha_fal              3145659
year                   3145659
epi_week               3145659
dtype: int64

In [ ]:
# Agregamos la información de sexo, edad y departamento de fallecimientos (columnas y)
# a las columnas de casos positivos (columnas x)
df['sexo_x'] = df['sexo_x'].combine_first(df['sexo_y'])
df['edad_x'] = df['edad_x'].combine_first(df['edad_y'])
df['departamento_x'] = df['departamento_x'].combine_first(df['departamento_y'])

In [ ]:
df.isnull().sum()  # Verificamos 'missing values'

id_persona                   0
fecha_resultado         105771
metododx                105771
sexo_x                       0
edad_x                       0
departamento_x               0
fecha_fallecimiento    3145659
edad_y                 3145659
sexo_y                 3145659
departamento_y         3145659
fallecido                    0
fecha_fal              3145659
year                   3145659
epi_week               3145659
dtype: int64

In [ ]:
df

,id_persona,fecha_resultado,metododx,sexo_x,edad_x,departamento_x,fecha_fallecimiento,edad_y,sexo_y,departamento_y,fallecido,fecha_fal,year,epi_week
0,13708801,20200309.0,2.0,2.0,15.0,10.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
1,2020432,20200310.0,2.0,1.0,46.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
2,7979960,20200311.0,2.0,2.0,21.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
3,5493803,20200311.0,2.0,2.0,28.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
4,35668643,20200312.0,2.0,2.0,39.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353122,513949,NaN,NaN,1.0,53.0,21.0,20220309.0,53.0,1.0,21.0,1,2022-03-09,2022.0,10.0
3353123,5783204,NaN,NaN,1.0,88.0,8.0,20220310.0,88.0,1.0,8.0,1,2022-03-10,2022.0,10.0
3353124,9290120,NaN,NaN,1.0,69.0,11.0,20220310.0,69.0,1.0,11.0,1,2022-03-10,2022.0,10.0
3353125,649346,NaN,NaN,2.0,68.0,18.0,20220310.0,68.0,2.0,18.0,1,2022-03-10,2022.0,10.0


In [ ]:
# Renombramos y ordenamos columnas
df = df.rename(columns = {'sexo_x': 'sexo', 'edad_x': 'edad', 'departamento_x': 'departamento'})
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'departamento',
         'fallecido', 'fecha_fallecimiento', 'fecha_fal', 'year', 'epi_week']]

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week
0,13708801,20200309.0,2.0,2.0,15.0,10.0,0,NaN,NaT,NaN,NaN
1,2020432,20200310.0,2.0,1.0,46.0,15.0,0,NaN,NaT,NaN,NaN
2,7979960,20200311.0,2.0,2.0,21.0,15.0,0,NaN,NaT,NaN,NaN
3,5493803,20200311.0,2.0,2.0,28.0,15.0,0,NaN,NaT,NaN,NaN
4,35668643,20200312.0,2.0,2.0,39.0,15.0,0,NaN,NaT,NaN,NaN


In [ ]:
# Obtener el número de días a partir del inicio de la pandemia en que una persona fallece
def days_between(d1, d2):
    '''Función que retorna el número de días entre 2 días dados'''
    return abs((d2 - d1).days)

fecha_min = df['fecha_fal'].min()
print(f'Primera fecha reportada de un fallecimiento en el dataset: {fecha_min}')
fecha_max = df['fecha_fal'].max()
print(f'Última fecha reportada de un falleccimiento el dataset: {fecha_max}')

# Obtener el número de días transcurridos a partir de la primera fecha reportada (fecha_min)
df['tiempo_fal'] = df['fecha_fal'].apply(lambda row : days_between(row, fecha_min)+1)

# Llenar 'nan values' con la última fecha del estudio
df['tiempo_fal'] = df['tiempo_fal'].fillna(df['tiempo_fal'].max()).astype('int')

Primera fecha reportada de un fallecimiento en el dataset: 2020-03-03 00:00:00
Última fecha reportada de un falleccimiento el dataset: 2022-03-10 00:00:00


**NOTA**: Se toma como fecha cero la de fallecimiento porque el primer reporte de caso confirmado se da luego del reporte primer reporte de fallecimiento por COVID-19.

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week,tiempo_fal
0,13708801,20200309.0,2.0,2.0,15.0,10.0,0,NaN,NaT,NaN,NaN,738
1,2020432,20200310.0,2.0,1.0,46.0,15.0,0,NaN,NaT,NaN,NaN,738
2,7979960,20200311.0,2.0,2.0,21.0,15.0,0,NaN,NaT,NaN,NaN,738
3,5493803,20200311.0,2.0,2.0,28.0,15.0,0,NaN,NaT,NaN,NaN,738
4,35668643,20200312.0,2.0,2.0,39.0,15.0,0,NaN,NaT,NaN,NaN,738


In [ ]:
df[df.fallecido == 1].head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week,tiempo_fal
5,36611013,20200312.0,2.0,1.0,78.0,15.0,1,20200319.0,2020-03-19,2020.0,12.0,17
9,36889651,20200312.0,2.0,2.0,75.0,15.0,1,20200320.0,2020-03-20,2020.0,12.0,18
51,36450848,20200315.0,2.0,1.0,96.0,15.0,1,20200329.0,2020-03-29,2020.0,14.0,27
62,36539210,20200316.0,2.0,1.0,71.0,7.0,1,20200414.0,2020-04-14,2020.0,16.0,43
106,36818328,20200318.0,2.0,2.0,63.0,15.0,1,20200429.0,2020-04-29,2020.0,18.0,58


In [ ]:
# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')

df['edad_cat'] = edad_cat(df, 'edad') 

In [ ]:
# Ordenar columnas
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat',
       'departamento', 'fallecido', 'fecha_fallecimiento', 'fecha_fal', 'year',
       'epi_week', 'tiempo_fal', ]]

In [ ]:
# Transformar 'data types'
df = df.astype({'sexo': 'int8', 'edad': 'int8', 'departamento': 'int8'})

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week,tiempo_fal
0,13708801,20200309.0,2.0,2,15,0,10,0,NaN,NaT,NaN,NaN,738
1,2020432,20200310.0,2.0,1,46,3,15,0,NaN,NaT,NaN,NaN,738
2,7979960,20200311.0,2.0,2,21,1,15,0,NaN,NaT,NaN,NaN,738
3,5493803,20200311.0,2.0,2,28,1,15,0,NaN,NaT,NaN,NaN,738
4,35668643,20200312.0,2.0,2,39,2,15,0,NaN,NaT,NaN,NaN,738


In [ ]:
# Missing values
df.isnull().sum()

id_persona                   0
fecha_resultado         105771
metododx                105771
sexo                         0
edad                         0
edad_cat                     0
departamento                 0
fallecido                    0
fecha_fallecimiento    3145659
fecha_fal              3145659
year                   3145659
epi_week               3145659
tiempo_fal                   0
dtype: int64

Las columnas de fecha resultado y metododx con datos faltantes se deben a que se agregaron personas del dataset de fallecidos que no se encontraban en el dataset de casos positivos, es decir nunca se hicieron pruebas para confirmar COVID-19, pero están registrados como causa de muerte COVID-19, evidentemente son casos postivos y se agregan al dataset.

#### 2.6.1 Tratar 'missing values' en fechas de fallecimiento

Como no existe año (year) y semana epidemiológica (epi_week) de fallecimiento para las personas que NO fallecieron, se decide llenar estos espacios con el año y semana epidemiológica última reportada en el dataset.

In [ ]:
# Tupla con año y semana epidemiológica última reportada en el dataset df
last_date = Week.fromdate(fecha_max).weektuple()
last_date

(2022, 10)

In [ ]:
df['year'].fillna(last_date[0], inplace=True)
df['epi_week'].fillna(last_date[1], inplace=True)

# Transformar 'data types'
df = df.astype({'year': 'int16', 'epi_week': 'int8'})

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week,tiempo_fal
0,13708801,20200309.0,2.0,2,15,0,10,0,NaN,NaT,2022,10,738
1,2020432,20200310.0,2.0,1,46,3,15,0,NaN,NaT,2022,10,738
2,7979960,20200311.0,2.0,2,21,1,15,0,NaN,NaT,2022,10,738
3,5493803,20200311.0,2.0,2,28,1,15,0,NaN,NaT,2022,10,738
4,35668643,20200312.0,2.0,2,39,2,15,0,NaN,NaT,2022,10,738


In [ ]:
# Missing values
df.isnull().sum()

id_persona                   0
fecha_resultado         105771
metododx                105771
sexo                         0
edad                         0
edad_cat                     0
departamento                 0
fallecido                    0
fecha_fallecimiento    3145659
fecha_fal              3145659
year                         0
epi_week                     0
tiempo_fal                   0
dtype: int64

### 2.7 Dataset de vacunados

#### 2.7.1 Leer dataset

In [ ]:
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
df_vac = pd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'id_vacuna': 'int8',
                                                                      'id_persona': 'int32'})

In [ ]:
df_vac.head()

,id_persona,fecha_vacunacion,id_vacuna,dosis
0,13474929,20211127,5,2
1,15456861,20220222,5,3
2,15428123,20211013,1,1
3,17787963,20211014,6,2
4,17873180,20220227,5,1


In [ ]:
# Renombrar y ordenar columnas
df_vac = df_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
df_vac = df_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

In [ ]:
df_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,13474929,2,20211127,5
1,15456861,3,20220222,5
2,15428123,1,20211013,1
3,17787963,2,20211014,6
4,17873180,1,20220227,5


In [ ]:
# TOTAL de vacunas aplicadas
len(df_vac)

63386865

#### 2.7.2 Separar información de dosis recibidas para juntarlos con nuestro dataframe principal (df)

In [ ]:
print(f'Total de dosis 1 administradas: {len(df_vac[df_vac.dosis == 1])}')
print(f'Total de dosis 2 administradas: {len(df_vac[df_vac.dosis == 2])}')
print(f'Total de dosis 3 administradas: {len(df_vac[df_vac.dosis == 3])}')
print(f'Total de 4 o más dosis administradas: {len(df_vac[df_vac.dosis >= 4])}')

Total de dosis 1 administradas: 27948872
Total de dosis 2 administradas: 24789831
Total de dosis 3 administradas: 10647688
Total de 4 o más dosis administradas: 474


In [ ]:
# Agregar información de dosis 1
df = df.merge(df_vac[df_vac.dosis == 1], on = 'id_persona', how = 'left')
df = df.rename(columns = {'dosis' : 'dosis_1', 
                          'fecha_vacuna': 'fecha_vacuna_1', 
                          'id_vacuna': 'id_vacuna_1'})

In [ ]:
# Agregar información de dosis 2
df = df.merge(df_vac[df_vac.dosis == 2], on = 'id_persona', how = 'left')
df = df.rename(columns = {'dosis' : 'dosis_2', 
                          'fecha_vacuna': 'fecha_vacuna_2', 
                          'id_vacuna': 'id_vacuna_2'})
df['dosis_2'].replace(2, 1, inplace = True)

In [81]:
# Agregar información de dosis 3
df = df.merge(df_vac[df_vac.dosis == 3], on = 'id_persona', how = 'left')
df = df.rename(columns = {'dosis' : 'dosis_3', 
                          'fecha_vacuna': 'fecha_vacuna_3', 
                          'id_vacuna': 'id_vacuna_3'})
df['dosis_3'].replace(3, 1, inplace = True)

In [82]:
# Verificamos que existen algunos duplicados al momento de añadir info de vacunación así que los eliminamos
len(df[df.id_persona.duplicated() == True])

26

In [83]:
# Eliminamos los duplicados
df = df.drop_duplicates('id_persona', keep='last') \
       .sort_values(by = ['tiempo_fal']) \
       .reset_index(drop = True)

In [84]:
# TOTAL de casos positivos más información de fallecimiento, más información de vacunación
len(df)

3353127

In [85]:
print(f'Personas con 1 dosis recibida en el dataset de casos positivos: {len(df[df.dosis_1 == 1])}')
print(f'Personas con 2 dosis recibidas en el dataset de casos positivos: {len(df[df.dosis_2 == 1])}')
print(f'Personas con 3 dosis recibidas en el dataset de casos positivos: {len(df[df.dosis_3 == 1])}')

Personas con 1 dosis recibida en el dataset de casos positivos: 2951538
Personas con 2 dosis recibidas en el dataset de casos positivos: 2824749
Personas con 3 dosis recibidas en el dataset de casos positivos: 1631547


In [86]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,...,tiempo_fal,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3
0,36527082,NaN,NaN,1,63,5,22,1,20200303.0,2020-03-03,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36401995,NaN,NaN,1,85,7,22,1,20200304.0,2020-03-04,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36853977,NaN,NaN,2,24,1,16,1,20200307.0,2020-03-07,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14179222,NaN,NaN,2,76,6,8,1,20200308.0,2020-03-08,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36614476,20200506.0,2.0,1,4,0,15,1,20200314.0,2020-03-14,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2.7.3 Rellenar espacios vacíos

In [88]:
# Crear una columna llamado 'vacunado' si recibe 2 o 3 dosis
df['vacunado'] = np.where((df['dosis_2'] == 1) | (df['dosis_3'] == 1), 1, 0).astype('int8')

In [89]:
# Convertir de float a int para ocupar menos espacio en memoria
df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

In [90]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,...,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3,vacunado
0,36527082,NaN,NaN,1,63,5,22,1,20200303.0,2020-03-03,...,0,NaN,0,0,NaN,0,0,NaN,0,0
1,36401995,NaN,NaN,1,85,7,22,1,20200304.0,2020-03-04,...,0,NaN,0,0,NaN,0,0,NaN,0,0
2,36853977,NaN,NaN,2,24,1,16,1,20200307.0,2020-03-07,...,0,NaN,0,0,NaN,0,0,NaN,0,0
3,14179222,NaN,NaN,2,76,6,8,1,20200308.0,2020-03-08,...,0,NaN,0,0,NaN,0,0,NaN,0,0
4,36614476,20200506.0,2.0,1,4,0,15,1,20200314.0,2020-03-14,...,0,NaN,0,0,NaN,0,0,NaN,0,0


In [91]:
# Verificamos columnas y data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353127 entries, 0 to 3353126
Data columns (total 23 columns):
 #   Column               Dtype         
---  ------               -----         
 0   id_persona           int32         
 1   fecha_resultado      float64       
 2   metododx             float64       
 3   sexo                 int8          
 4   edad                 int8          
 5   edad_cat             int8          
 6   departamento         int8          
 7   fallecido            int8          
 8   fecha_fallecimiento  float64       
 9   fecha_fal            datetime64[ns]
 10  year                 int16         
 11  epi_week             int8          
 12  tiempo_fal           int64         
 13  dosis_1              int8          
 14  fecha_vacuna_1       float64       
 15  id_vacuna_1          int8          
 16  dosis_2              int8          
 17  fecha_vacuna_2       float64       
 18  id_vacuna_2          int8          
 19  dosis_3              

## 3. Ordenar y Guardar el df

In [93]:
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat',
       'departamento', 'fallecido', 'fecha_fallecimiento', 'fecha_fal', 'year',
       'epi_week', 'tiempo_fal', 'vacunado', 'dosis_1', 'fecha_vacuna_1', 'id_vacuna_1',
       'dosis_2', 'fecha_vacuna_2', 'id_vacuna_2', 'dosis_3', 'fecha_vacuna_3',
       'id_vacuna_3']]

In [96]:
# Guardamos en un csv
file_name = 'DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv'
df.to_csv('/content/drive/MyDrive/GitHub/covid19-peru/Data/' + file_name,  index=False)